In [53]:
import re
keywords = ['IF' , 'WHILE' , 'ELSE' , 'PRINTF']
def lexer(code):
    token_specification = [
        ('NUMBER',   r'\d+(\.\d*)?'),       
        ('IDENT',    r'[A-Za-z_]\w*'),      
        ('ASSIGN',   r'='),
        ('PLUS_ASSIGN', r'\+='),
        ("SUB_ASSIGN", r'-='),                 
        ('END',      r';'),                 
        ('ADD',      r'\+'),               
        ('SUB',      r'-'),                 
        ('MUL',      r'\*'),              
        ('DIV',      r'/'),                
        ('LPAREN',   r'\('),             
        ('RPAREN',   r'\)'),              
        ('LBRACE',   r'\{'),              
        ('RBRACE',   r'\}'),                               
        ('GT',       r'>'),                
        ('LT',       r'<'),                
        ('SKIP',     r'[ \t\n]+|"%d" ,'),         
        ('MISMATCH', r'.'),                
    ]
    tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
    tokens = []
    line_num = 1
    for mo in re.finditer(tok_regex, code):
        kind = mo.lastgroup
        value = mo.group()
        if kind == 'NUMBER':
            value = float(value) if '.' in value else int(value)
            tokens.append((kind, value, line_num))
        elif kind == 'IDENT':
            tokens.append((kind, value, line_num))
        elif kind == 'SKIP':
            if '\n' in value:
                line_num += value.count('\n')
            continue
        elif kind == 'MISMATCH':
            raise RuntimeError(f'{value!r} unexpected on line {line_num}')
        else:
            tokens.append((kind, value, line_num))
    return tokens

In [54]:
class Parser:
    def __init__(self,token):
        self.tokens = token
        self.pos = 0
        self.symtable = []
        
    def parse(self):
        statements = []
        while self.current_token():
            statements.append(self.statement())
            print(statements)
        return statements

    def statement(self):
        if self.current_token() == 'IF':
            return self.if_statement()
        elif self.current_token() == 'WHILE':
            return self.while_statement()
        elif self.current_token() == 'PRINTF':
            return self.printf_statement()
        elif self.current_token() == 'IDENT' and self.tokens[self.pos][1] == 'int' and self.peek_token() == 'IDENT':
            return self.declaration()
        elif self.current_token() == 'IDENT' and self.peek_token() == 'ASSIGN':
            return self.assignment()
        elif self.current_token() == 'IDENT' and self.peek_token() == 'PLUS_ASSIGN':
            return self.p_assign();
        elif self.current_token() == 'IDENT' and self.peek_token() == 'SUB_ASSIGN':
            return self.s_assign();
        elif self.current_token() == 'IDENT' and self.current_token() not in keywords:
            raise RuntimeError(f"Unknown keywords {self.tokens[self.pos][1]} at line {self.tokens[self.pos][1]}")
        
    def declaration(self):
        self.next_token()
        var_name = self.match('IDENT')[1]
        if var_name not in self.symtable:
            self.symtable.append(var_name)
        else:
            raise RuntimeError(f"Previous defined variable {self.tokens[self.pos-1][1]} at line {self.current_line()}")
        self.match('ASSIGN')
        expr = self.expression()
        self.match('END')
        return ('DECLARE', var_name, expr)
    
    def assignment(self):
        var_name = self.match('IDENT')[1]
        if var_name not in self.symtable:
            raise RuntimeError(f"At line {self.tokens[self.pos][2]}, variable {var_name} is not defined")
        self.match('ASSIGN')
        expr = self.expression()
        self.match('END')
        return ('ASSIGN', var_name, expr)

    def if_statement(self):
        self.match('IF')
        self.match('LPAREN')
        condition = self.expression()
        self.match('RPAREN')
        self.match('LBRACE')
        then_branch = []
        while self.current_token() != 'RBRACE':
            then_branch.append(self.statement())
        self.match('RBRACE')
        return ('IF', condition, then_branch)
    
    def while_statement(self):
        self.match('WHILE')
        self.match('LPAREN')
        condition = self.expression()
        self.match('RPAREN')
        self.match('LBRACE')
        body = []
        while self.current_token() != 'RBRACE':
            body.append(self.statement())
        self.match('RBRACE')
        return ('WHILE', condition, body)
    
    def printf_statement(self):
        self.match('PRINTF')
        self.match('LPAREN')
        var_name = self.match('IDENT')[1]
        self.match('RPAREN')
        self.match('END')
        return ('PRINTF', var_name)
    
    def p_assign(self):
        var_name = self.tokens[self.pos][1]
        if var_name not in self.symtable:
            raise RuntimeError(f"At line {self.tokens[self.pos][2]}, variable {var_name} is not defined")
        self.next_token()
        self.match('PLUS_ASSIGN')
        expr = self.expression()
        self.match('END')
        return ('PLUS_ASSIGN', var_name, expr)
        
    def s_assign(self):
        var_name = self.tokens[self.pos][1]
        if var_name not in self.symtable:
            raise RuntimeError(f"At line {self.tokens[self.pos][2]}, variable {var_name} is not defined")
        self.next_token()
        self.match('SUB_ASSIGN')
        expr = self.expression()
        self.match('END')
        return ('SUB_ASSIGN', var_name, expr)
    
    def expression(self):
        node = self.term()
        while self.current_token() in ('ADD', 'SUB', 'GT', 'LT'):
            token = self.current_token()
            self.next_token()
            if token == 'ADD':
                node = ('ADD', node, self.term())
            elif token == 'SUB':
                node = ('SUB', node, self.term())
            elif token == 'GT':
                node = ('GT', node, self.term())
            elif token == 'LT':
                node = ('LT', node, self.term())
        return node

    def term(self):
        node = self.factor()
        while self.current_token() in ('MUL', 'DIV'):
            token = self.current_token()
            self.next_token()
            if token == 'MUL':
                node = ('MUL', node, self.factor())
            elif token == 'DIV':
                node = ('DIV', node, self.factor())
        return node

    def factor(self):
        token = self.current_token()
        if token == 'NUMBER':
            self.next_token()
            return ('NUMBER', self.tokens[self.pos - 1][1])
        elif token == 'IDENT':
            if self.tokens[self.pos][1] not in self.symtable:
                raise RuntimeError(f"At line {self.tokens[self.pos][2]}, variable {self.tokens[self.pos][1]} is not defined")
            self.next_token()
            return ('IDENT', self.tokens[self.pos - 1][1])
        elif token == 'LPAREN':
            self.next_token()
            node = self.expression()
            self.match('RPAREN')
            return node

    def current_token(self):
        if self.pos < len(self.tokens):
            if self.tokens[self.pos][1] == 'if':
                return 'IF'
            elif self.tokens[self.pos][1] == 'while':
                return 'WHILE'
            elif self.tokens[self.pos][1] == 'printf':
                return 'PRINTF'
            else:
                return self.tokens[self.pos][0]
        return None

    def peek_token(self):
        if self.pos + 1 < len(self.tokens):
            if self.tokens[self.pos+1][1] == 'if':
                return 'IF'
            elif self.tokens[self.pos+1][1] == 'while':
                return 'WHILE'
            elif self.tokens[self.pos+1][1] == 'printf':
                return 'PRINTF'
            return self.tokens[self.pos + 1][0]
        return None

    def next_token(self):
        self.pos += 1
        if(self.pos > len(self.tokens)):
            self.pos = len(self.tokens)-1

    def current_line(self):
        return self.tokens[self.pos][2]

    def match(self, token_type):
        if self.current_token() != token_type:
            raise RuntimeError(f'Expected {token_type} but got {self.current_token(), self.tokens[self.pos][1]} at line {self.current_line()}')
        if self.tokens[self.pos][1] == 'if':
            token = 'IF'
        elif self.tokens[self.pos][1] == 'while':
            token = 'WHILE'
        elif self.tokens[self.pos][1] == 'printf':
            token = 'PRINTF'
        else:
            token = self.tokens[self.pos]
        self.next_token()
        return token
    
def generate_code(statements):
    code = []
    for stmt in statements:
        if stmt[0] == 'ASSIGN' or stmt[0] == 'DECLARE':
            code.append(f"{stmt[1]} = {generate_expr(stmt[2])}")
        elif stmt[0] == 'IF':
            code.append(f"if {generate_expr(stmt[1])}:")
            for sub_stmt in stmt[2]:
                code.append(f"    {generate_code([sub_stmt])}")
        elif stmt[0] == 'WHILE':
            code.append(f"while {generate_expr(stmt[1])}:")
            for sub_stmt in stmt[2]:
                code.append(f"    {generate_code([sub_stmt])}")
        elif stmt[0] == 'PRINTF':
            code.append(f"print({stmt[1]})")
        elif stmt[0] == 'PLUS_ASSIGN':
            code.append(f"{stmt[1]} += {generate_expr(stmt[2])}")
        elif stmt[0] == 'SUB_ASSIGN':
            code.append(f"{stmt[1]} -= {generate_expr(stmt[2])}")    
    return '\n'.join(code)

def generate_expr(node):
    if node[0] == 'NUMBER':
        return str(node[1])
    elif node[0] == 'IDENT':
        return node[1]
    elif node[0] == 'ADD':
        return f"({generate_expr(node[1])} + {generate_expr(node[2])})"
    elif node[0] == 'SUB':
        return f"({generate_expr(node[1])} - {generate_expr(node[2])})"
    elif node[0] == 'MUL':
        return f"({generate_expr(node[1])} * {generate_expr(node[2])})"
    elif node[0] == 'DIV':
        return f"({generate_expr(node[1])} / {generate_expr(node[2])})"
    elif node[0] == 'GT':
        return f"({generate_expr(node[1])} > {generate_expr(node[2])})"
    elif node[0] == 'LT':
        return f"({generate_expr(node[1])} < {generate_expr(node[2])})"


def compile_code(code):
    tokens = lexer(code)
    #print(tokens)
    parser = Parser(tokens)
    statements = parser.parse()
    return generate_code(statements)

In [55]:
file = open('test.c' , 'r')
code = file.read()
print(code)
compiled = compile_code(code)
print("----")
print(compiled)
print("----")
exec(compiled)
file.close()

int sum = 0;
int i = 1;
while(i < 10){
    sum += i;
    printf(sum);
    i += 1;
}
[('DECLARE', 'sum', ('NUMBER', 0))]
[('DECLARE', 'sum', ('NUMBER', 0)), ('DECLARE', 'i', ('NUMBER', 1))]
[('DECLARE', 'sum', ('NUMBER', 0)), ('DECLARE', 'i', ('NUMBER', 1)), ('WHILE', ('LT', ('IDENT', 'i'), ('NUMBER', 10)), [('PLUS_ASSIGN', 'sum', ('IDENT', 'i')), ('PRINTF', 'sum'), ('PLUS_ASSIGN', 'i', ('NUMBER', 1))])]
----
sum = 0
i = 1
while (i < 10):
    sum += i
    print(sum)
    i += 1
----
1
3
6
10
15
21
28
36
45
